### IMAT

> 

In [57]:
import sys
import os
import csv
import pandas as pd
import seaborn as sns
import numpy as np
#np.set_printoptions(threshold=np.nan)
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import RandomizedSearchCV
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
from scipy import stats
import matplotlib.font_manager
from sklearn import svm
import pickle
from sklearn.decomposition import PCA



### Reading data

In [58]:
#data = pd.read_csv('Final_Ecomm_data_19thJan.csv')#,index_col=0)
data = pd.read_csv('IMAT_train.csv')#,index_col=0)

In [59]:
data.head(10)

,src_uid,trn_uid,purpose,destination,year,spare_time,degree,branch,gender,hostel,state,score,label
0,2,4,0.0,6.847826,6,1.294118,5,4,0.0,0.0,0,23.141944,1
1,2,7,0.0,4.239130,6,3.558824,5,0,2.5,2.5,0,23.797954,1
2,2,25,0.0,5.217391,0,0.647059,5,0,0.0,0.0,0,10.864450,0
3,29,39,0.0,2.608696,0,4.852941,5,0,0.0,0.0,0,12.461637,0
4,29,47,0.0,5.217391,0,5.500000,0,4,2.5,0.0,0,17.217391,1
5,29,48,0.0,2.608696,6,0.970588,0,0,2.5,0.0,0,12.079284,0
6,53,56,7.5,5.869565,0,2.264706,0,0,2.5,0.0,0,18.134271,1
7,53,66,0.0,3.586957,0,0.647059,0,4,2.5,0.0,0,10.734015,0
8,53,69,0.0,3.586957,0,5.500000,0,0,2.5,0.0,0,11.586957,0
9,73,75,0.0,2.608696,0,1.294118,5,4,0.0,0.0,0,12.902813,0


In [60]:
data.tail(10)

,src_uid,trn_uid,purpose,destination,year,spare_time,degree,branch,gender,hostel,state,score,label
62,15,41,0.0,3.586957,0,1.294118,5,4,2.5,0.0,0,16.381074,1
63,42,43,0.0,5.869565,6,0.647059,5,4,2.5,0.0,1,25.016624,1
64,42,46,7.5,4.891304,0,0.647059,0,0,0.0,0.0,0,13.038363,1
65,42,55,0.0,7.500000,0,1.941176,0,0,2.5,0.0,0,11.941176,0
66,60,62,0.0,4.891304,6,1.941176,5,0,2.5,0.0,0,20.332481,1
67,60,77,0.0,2.608696,0,1.941176,5,4,0.0,0.0,0,13.549872,1
68,60,80,0.0,4.891304,0,2.911765,5,0,0.0,0.0,0,12.803069,0
69,87,89,0.0,5.869565,6,0.970588,5,0,0.0,0.0,0,17.840153,1
70,87,94,7.5,4.891304,0,1.294118,5,0,0.0,0.0,0,18.685422,1
71,87,96,0.0,1.956522,0,2.911765,5,0,2.5,0.0,0,12.368286,0


In [61]:
data.shape

(72, 13)

#### Pre-processing data for fitting

In [63]:
# X -- Feature vectors - Type - Dataframe

#to_keep = ['purpose', 'destination' ,'year','spare_time','degree','branch','gender','hostel','state']
y=data['label']

#X = data[to_keep]
#Drop all unecessary data
to_drop = ['src_uid','trn_uid','score','label']
X = data.drop(to_drop,axis=1)


In [64]:
#Feature vector 
X.head(8)
#X.ix[:5,4:]

,purpose,destination,year,spare_time,degree,branch,gender,hostel,state
0,0.0,6.847826,6,1.294118,5,4,0.0,0.0,0
1,0.0,4.239130,6,3.558824,5,0,2.5,2.5,0
2,0.0,5.217391,0,0.647059,5,0,0.0,0.0,0
3,0.0,2.608696,0,4.852941,5,0,0.0,0.0,0
4,0.0,5.217391,0,5.500000,0,4,2.5,0.0,0
5,0.0,2.608696,6,0.970588,0,0,2.5,0.0,0
6,7.5,5.869565,0,2.264706,0,0,2.5,0.0,0
7,0.0,3.586957,0,0.647059,0,4,2.5,0.0,0


In [18]:
# y -- Target values - Type - Series
y=X['Default_Flag']

to_drop = ['Default_Flag','Cibil_Score','SD_Score','Due_30_Days','Due_60_Days','Due_90_Days']

X = X.drop(to_drop,axis=1)

print type(X)
print type(y)
#print len(y),len(X)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


### Hyper-Parameter Tuning

* ### K-Fold Cross validation

In [74]:
#A demo display of cross validation with no of neurons in hidden layer = 27

clf = MLPClassifier(hidden_layer_sizes=(7))
scores = cross_val_score(clf, X, y, cv=3, scoring='accuracy')
print scores

[ 0.75        0.66666667  0.70833333]


In [94]:
# use average accuracy as an estimate of out-of-sample accuracy
print scores.mean()

0.763888888889


In [95]:
# search for an optimal value of Number of neurons in hidden layer-1 for Multiple Layer Perceptron
#
#X.shape[1] holds number of features
n_range = range(2, X.shape[1])
print n_range
n_scores = []
for n in n_range:
    clf = MLPClassifier(hidden_layer_sizes=n)
    scores = cross_val_score(clf, X, y, cv=3, scoring='accuracy')
    print scores
    n_scores.append(scores.mean())
print n_scores

[2, 3, 4, 5, 6, 7, 8]
[ 0.79166667  0.70833333  0.70833333]
[ 0.70833333  0.75        0.91666667]
[ 0.75        0.70833333  0.70833333]
[ 0.70833333  0.66666667  0.875     ]
[ 0.75        0.70833333  0.79166667]
[ 0.75        0.70833333  0.58333333]
[ 0.75        0.83333333  0.875     ]
[0.73611111111111116, 0.79166666666666663, 0.72222222222222232, 0.75, 0.75, 0.68055555555555569, 0.81944444444444453]


In [96]:
## DANGER - DONT Re-RUN ; Use Pickle
cross_val_single_layer = open('cross_val_single_layer.pickle', 'wb')

# Pickle the list using the highest protocol available.
pickle.dump(n_scores, cross_val_single_layer, -1) ###

cross_val_single_layer.close()

In [97]:
#Loading data from pickle file directly
n_range = range(2, X.shape[1])
pkl_file_cross_val = open('cross_val_single_layer.pickle', 'rb')
n_scores = pickle.load(pkl_file_cross_val)
pkl_file_cross_val.close()

In [98]:
# use average accuracy as an estimate of out-of-sample accuracy
print sum(n_scores)/float(len(n_scores))

0.75


#### Machine learning - Modeling 

In [99]:
# default split is 75% for training and 25% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y)

#Prediction of test and training data

#n for which accuracy is maximum in single hidden layer using cross validation
n=n_range[n_scores.index(max(n_scores))]
#print n

#SET CLF AGAIN TO BEST N FIRST
clf= MLPClassifier(hidden_layer_sizes=n)
clf.fit(X_train,y_train)

y_pred_test = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
y_pred_proba_test = clf.predict_proba(X_test)
y_pred_proba_train = clf.predict_proba(X_train)


* #### Accuracy

In [101]:
print "Test Accuracy (Single hidden layer CV) - ", metrics.accuracy_score(y_test, y_pred_test)
print "Training Accuracy (Single hidden layer CV) - ", metrics.accuracy_score(y_train, y_pred_train)


Test Accuracy (Single hidden layer CV) -  0.722222222222
Training Accuracy (Single hidden layer CV) -  0.833333333333


In [102]:
cwd = os.getcwd()
fName = "{}/nn_classifier.pkl".format(cwd)
print("Saving classifier to '{}'".format(fName))
with open(fName, 'w') as f:
    pickle.dump(clf, f)

Saving classifier to '/Users/paawan/Desktop/nn_classifier.pkl'
